In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype

In [2]:
df = pd.read_csv('results.csv')
df.head()

,Price,Reviews,Rating,Brand,Operating System,CPU manufacturer,Screen size,Computer memory size,Processor count,Graphics Chipset Brand,...,Wireless communication technology,Human interface input,Connectivity Type,Graphics co-processor,Battery cell composition,Item dimensions L x W x H,Cellular technology,Other camera features,Software included,Wireless communication standard
0,£667.69,218 ratings,4.6 out of 5 stars,HUAWEI,Windows 10 Home,Intel,13 Inches,8 GB,8.0,NVIDIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,£399.99,98 ratings,3.8 out of 5 stars,TECLAST,Windows 10,Intel Celeron Processor N4100,11.6 Inches,8 GB,4.0,Intel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,£228.99,"2,586 ratings",4.0 out of 5 stars,ASUS,Windows 10 Home,Intel,14,4 GB,2.0,Intel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,£499.00,40 ratings,4.5 out of 5 stars,Dell,Windows 10,AMD,15.6 Inches,8 GB,1.0,AMD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,£229.49,715 ratings,4.2 out of 5 stars,Jumper,Windows 10,Intel,13.3 Inches,4 GB,2.0,Intel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


First thing to notice is the columns with the large amount of null, so find the percentage null in each column

In [3]:
(df.isnull().sum()/df.shape[0])*100 

Price                                 0.000000
Reviews                               0.000000
Rating                                9.302326
Brand                                 0.000000
Operating System                      5.813953
CPU manufacturer                      3.488372
Screen size                           1.744186
Computer memory size                  0.581395
Processor count                       2.325581
Graphics Chipset Brand               19.476744
Series                               50.290698
CPU model                             4.069767
Colour                               77.906977
Hard disk size                       24.709302
Resolution                           49.418605
Item weight                          93.604651
Form Factor                          97.383721
Hard disk description                93.604651
Operating system                     95.058140
Graphics processor manufacturer      97.093023
Model name                           96.802326
RAM memory te

Drop any column where the percentage null is greater than 40 percent

In [4]:
colsToDrop = df.columns[(df.isnull().sum()/df.shape[0])*100 > 40]
df.drop(colsToDrop,axis = 1,inplace =  True)

In [5]:
df = df.replace(',','', regex=True)
# Remove £ sign so we can convert to numeric types
df = df.replace('£','',regex = True)

df = df.replace('ratings','',regex = True)
df = df.replace('rating','',regex = True)
df = df.replace('out of 5 stars','',regex = True)

In [6]:
df['Operating System'].unique()

array(['Windows 10 Home', 'Windows 10', 'Windows', 'Windows 10 S',
       'Android 10', 'Windows 10 Pro', 'Chrome OS', nan, 'Android',
       'Mac OS X', 'Windows 7', 'Google Chrome'], dtype=object)

Obviously Google Chrome is not an operating system so we will change that to NaN. Secondly Android 10 is the most version of Android so it would make sense that when one defaults to Android it is the same as Android 10 so we will merge them. Windows 10 Home is the same as Windows 10. We will merge Windows 10 Home, Windows 10 and Windows 7 into Windows. 

In [7]:
df[df['Operating System'] == 'Windows 7']

,Price,Reviews,Rating,Brand,Operating System,CPU manufacturer,Screen size,Computer memory size,Processor count,Graphics Chipset Brand,CPU model,Hard disk size
296,299.99,61,4.2,X230,Windows 7,Intel,12.5 Inches,4 GB,1.0,NaN,Core i5,320 GB


In [8]:
df['Operating System'] = df['Operating System'].replace(['Windows 7','Windows 10','Windows 10 Home'],'Windows')
df['Operating System'] = df['Operating System'].replace('Android 10','Android')

In [9]:
df['CPU manufacturer'].unique()

array(['Intel', 'Intel Celeron Processor N4100', 'AMD', 'MediaTek',
       'Apple', nan, 'Intel Core i5-7300U', 'A33', 'VIA'], dtype=object)

In [10]:
# Convert intel processor types to just intel
df['CPU manufacturer'] = df['CPU manufacturer'].replace([ 'Intel Celeron Processor N4100','Intel Core i5-7300U'],'Intel')

In [11]:
df['Screen size'].unique()

array(['13 Inches', '11.6 Inches', '14', '15.6 Inches', '13.3 Inches',
       '14 Inches', '12.45 Inches', '10.1 Inches', '14.1 Inches',
       '17.3 Inches', '15.6', '15 Inches', nan, '10.1 Feet',
       '13.9 Inches', '12.5 Inches', '13.3', '35.6 Centimetres',
       '27 Inches', '17 Inches', '15.6 Centimetres', '15.4 Inches',
       '10 Inches', '6 Inches', '21.5 Inches', '11 Inches', '13.5 Inches',
       '13.4 Inches', '12 Inches', '16 Inches', '34.39 Centimetres'],
      dtype=object)

Want to convert screen size to an float so remove the inches and find which data is in centimeters and convert to inches

In [12]:
df['Screen size'] = df['Screen size'].str.replace('Inches','')
df[df['Screen size'].str.contains('Centimetres',na = False)]

,Price,Reviews,Rating,Brand,Operating System,CPU manufacturer,Screen size,Computer memory size,Processor count,Graphics Chipset Brand,CPU model,Hard disk size
121,349.00,893,3.6,HP,Windows,Intel,35.6 Centimetres,4 GB,2.0,Intel,Celeron,32 GB
151,1198.99,267,4.5,Razer,Windows,Intel,15.6 Centimetres,256 GB,6.0,Intel,Core i7,16 GB
324,1446.82,96,4.4,Microsoft,Windows,Intel,34.39 Centimetres,8 GB,4.0,Intel,Core i7,NaN


There are only three rows so just replace them manually by looking up the models on Amazon

In [13]:
df['Screen size'] = df['Screen size'].str.replace('35.6 Centimetres','14')
df['Screen size'] = df['Screen size'].str.replace('34.39 Centimetres','13.5')
# 15.6 Centimeters is too small for a laptop, checked this value on Amazon it is 15.6 inches 
df['Screen size'] = df['Screen size'].str.replace('15.6 Centimetres','15.6')
# Same as above
df['Screen size'] = df['Screen size'].str.replace('Feet','')

Moving onto RAM which is called computer memory size, notice that we have values which are obviously too large.  1 TB is obviously the hard disk size and not the RAM so replace the obviously false data manually

In [14]:
df['Computer memory size'].unique()

array(['8 GB', '4 GB', '6 GB', '2 GB', '16 GB', '512 GB', '1 TB', '12 GB',
       nan, '256 GB', '1 GB', '32 GB', '16'], dtype=object)

In [15]:
df[df['Computer memory size'] == '512 GB']


,Price,Reviews,Rating,Brand,Operating System,CPU manufacturer,Screen size,Computer memory size,Processor count,Graphics Chipset Brand,CPU model,Hard disk size
59,1555.00,817,4.7,Apple,NaN,Intel,13,512 GB,4.0,NaN,Core i5,NaN
146,2099.00,102,4.7,Apple,NaN,Intel,27,512 GB,8.0,NaN,Intel Core i7,NaN
220,2399.00,0,NaN,MSI,Windows 10 Pro,Intel,17.3,512 GB,8.0,NVIDIA,Core i7,512 GB
277,899.99,1,5.0,Lenovo,Windows 10 Pro,NaN,14,512 GB,1.0,NaN,Core i7,512 GB


In [16]:
df.at[59,'Computer memory size'] = '16'
df.at[146,'Computer memory size'] = '8'

df.at[59,'Hard disk size'] = '512 GB'
df.at[146,'Hard disk size'] = '512 GB '

df.at[220,'Computer memory size'] = '16'
df.at[277,'Computer memory size'] = '8'

In [17]:
df[df['Computer memory size'] == '1 TB']

,Price,Reviews,Rating,Brand,Operating System,CPU manufacturer,Screen size,Computer memory size,Processor count,Graphics Chipset Brand,CPU model,Hard disk size
74,1099.97,5,4.5,MSI,Windows,Intel,15.6,1 TB,6.0,NVIDIA,Core i7,1 TB
106,1099.97,5,4.5,MSI,Windows,Intel,15.6,1 TB,6.0,NVIDIA,Core i7,1 TB
198,1898.99,0,NaN,MSI,Windows,Intel,15.6,1 TB,8.0,NVIDIA,Core i7,1 TB


In [18]:
df.at[74,'Computer memory size'] = '16'
df.at[106,'Computer memory size'] = '16'
df.at[198,'Computer memory size'] = '16'
df['Computer memory size'] = df['Computer memory size'].str.replace('GB','').astype(np.float).astype("Int32")

In [19]:
df['Processor count'].unique()

array([ 8.,  4.,  2.,  1., nan,  6.,  3.,  7.])

In [20]:
df['Graphics Chipset Brand'].unique()
df['Graphics Chipset Brand'] = df['Graphics Chipset Brand'].str.replace('NVIDIA','Nvidia')

Obviously there are such a large amount of different processors lets see if we can combine them to smooth the analysis

In [21]:
df['CPU model'].value_counts()

Celeron                           70
Celeron N                         50
Core i5                           38
Core i7                           30
Core i5-10210U                    23
Celeron N3350                     13
Pentium                            6
Celeron J3455                      6
None                               6
Intel Core i5                      6
Core i5 Family                     6
Celeron N3450                      5
Atom Z8350                         4
Celeron G1610T                     4
Core i5 6200U                      4
Celeron 2955U                      3
5X86                               3
Ryzen 3 3250U                      3
Core i3-1005G1                     3
Core i5 3340M                      3
Celeron N4020                      2
A33 ARM Cortexâ¢ A7 Quad Core     2
AMD A4                             2
A-Series                           2
AMD Ryzen 7                        2
Ryzen 5 3500U                      2
Core_m                             2
C

In [22]:
# Replace all intel models with the family so Core i5 5300U becomes Core i% 
df['CPU model'] = df['CPU model'].str.replace(r'(^.*Core i3.*$)', 'Core i3')
df['CPU model'] = df['CPU model'].str.replace(r'(^.*Core i5.*$)', 'Core i5')
df['CPU model'] = df['CPU model'].str.replace(r'(^.*Core i7.*$)', 'Core i7')
df['CPU model'] = df['CPU model'].str.replace(r'(^.*Pentium.*$)', 'Pentium')
#Since there are so few Ryzen combine them into one category
df['CPU model'] = df['CPU model'].str.replace(r'(^.*Ryzen.*$)', 'Ryzen')
df['CPU model'] = df['CPU model'].str.replace(r'(^.*AMD.*$)', 'Ryzen')

Convert any CPU model which has less than 5 items into Other

In [23]:
counts = df['CPU model'].value_counts() 
idx = counts[counts.lt(5)].index
df.loc[df['CPU model'].isin(idx), 'CPU model'] = 'Other'

In [24]:
# A computer has to have a cpu so if it has None down just change it to Other
df['CPU model'] = df['CPU model'].str.replace('None','Other')

Convert types

In [25]:
# 1 TB is 1000 GB 
df['Hard disk size'] = df['Hard disk size'].str.replace('1 TB', '1000')
df['Hard disk size'] = df['Hard disk size'].str.replace('GB','').astype(np.float).astype("Int32")
df['Reviews'] = df['Reviews'].astype(np.float).astype("Int32")
df['Screen size'] = df['Screen size'].astype(np.float)
df['Price'] = df['Price'].astype(np.float)
df['Rating'] = df['Rating'].astype(np.float)

In [26]:
# Make sure all types are right
df.dtypes

Price                     float64
Reviews                     Int32
Rating                    float64
Brand                      object
Operating System           object
CPU manufacturer           object
Screen size               float64
Computer memory size        Int32
Processor count           float64
Graphics Chipset Brand     object
CPU model                  object
Hard disk size              Int32
dtype: object

For every column if the type is numeric fill the null with the average type if the column is a string then just write the word Unknown for Null values

In [27]:
for col in df.columns:
    if(is_numeric_dtype(df[col])):
        print(col)
        avg  = int(df[col].mean())
        df[col].fillna(avg,inplace=True)
        
    if(is_string_dtype(df[col])):
        df[col].fillna('Unknown ',inplace = True)

Price
Reviews
Rating
Screen size
Computer memory size
Processor count
Hard disk size


In [28]:
df.to_pickle("./cleaned_data")